# 🚗 Smart Traffic System - Model Training on Google Colab

**Goal:** Train XGBoost + LightGBM + Prophet models on Google Colab (Free T4 GPU)

**Steps:**
1. Mount Google Drive
2. Load data from CSV
3. Feature engineering
4. Train 3 models
5. Save models to Drive
6. Download to local machine

---

## 📦 STEP 1: Setup & Install Libraries

In [ ]:
# Install required packages
!pip install -q xgboost lightgbm prophet scikit-learn pandas numpy matplotlib seaborn plotly joblib

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score, classification_report
import xgboost as xgb
import lightgbm as lgb
from prophet import Prophet
import joblib

print("✅ Libraries imported successfully")
print(f"XGBoost version: {xgb.__version__}")
print(f"LightGBM version: {lgb.__version__}")

## 📁 STEP 2: Mount Google Drive & Load Data

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to your data
DATA_PATH = '/content/drive/MyDrive/SmartTraffic/data/traffic_data_for_training.csv'
MODEL_OUTPUT_PATH = '/content/drive/MyDrive/SmartTraffic/models/'

In [ ]:
# Load data
print("📊 Loading traffic data...")
df = pd.read_csv(DATA_PATH, parse_dates=['DateObservedFrom', 'DateObservedTo'])

print(f"✅ Loaded {len(df)} records")
print(f"\n📋 Columns: {df.columns.tolist()}")
print(f"\n📈 Shape: {df.shape}")
df.head()

## 🔧 STEP 3: Feature Engineering

In [ ]:
def create_features(df):
    """
    Create 18+ features for ML models
    """
    df = df.copy()
    
    # 1. Time features
    df['hour'] = df['DateObservedFrom'].dt.hour
    df['day_of_week'] = df['DateObservedFrom'].dt.dayofweek  # 0=Monday
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['is_rush_hour'] = ((df['hour'] >= 7) & (df['hour'] <= 9) | 
                           (df['hour'] >= 17) & (df['hour'] <= 19)).astype(int)
    
    # 2. Sort by segment and time
    df = df.sort_values(['RefRoadSegment', 'DateObservedFrom']).reset_index(drop=True)
    
    # 3. Lag features (previous values)
    df['speed_lag_1'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].shift(1)
    df['speed_lag_2'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].shift(2)
    df['speed_lag_3'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].shift(3)
    
    df['intensity_lag_1'] = df.groupby('RefRoadSegment')['Intensity'].shift(1)
    
    # 4. Rolling statistics
    df['speed_rolling_mean_6'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].rolling(6, min_periods=1).mean().reset_index(0, drop=True)
    df['speed_rolling_mean_12'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].rolling(12, min_periods=1).mean().reset_index(0, drop=True)
    df['speed_rolling_std_6'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].rolling(6, min_periods=1).std().reset_index(0, drop=True)
    
    df['intensity_rolling_mean_6'] = df.groupby('RefRoadSegment')['Intensity'].rolling(6, min_periods=1).mean().reset_index(0, drop=True)
    
    # 5. Difference features
    df['speed_diff'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].diff()
    df['intensity_diff'] = df.groupby('RefRoadSegment')['Intensity'].diff()
    
    # 6. Ratio features
    df['speed_to_max_ratio'] = df['AverageVehicleSpeed'] / df['MaximumAllowedSpeed']
    
    # 7. Segment encoding (one-hot)
    segment_dummies = pd.get_dummies(df['RefRoadSegment'], prefix='segment')
    df = pd.concat([df, segment_dummies], axis=1)
    
    # Fill NaN values
    df = df.fillna(method='bfill').fillna(method='ffill').fillna(0)
    
    print(f"✅ Created features. New shape: {df.shape}")
    return df

# Apply feature engineering
df_featured = create_features(df)
print("\n📊 Sample features:")
df_featured[['AverageVehicleSpeed', 'hour', 'is_rush_hour', 'speed_lag_1', 'speed_rolling_mean_6']].head(10)

## 🎯 STEP 4: Prepare Train/Test Split

In [ ]:
# Feature columns for ML
feature_cols = [
    'Intensity', 'Occupancy', 'TotalLaneNumber', 'MaximumAllowedSpeed',
    'hour', 'day_of_week', 'is_weekend', 'is_rush_hour',
    'speed_lag_1', 'speed_lag_2', 'speed_lag_3',
    'intensity_lag_1',
    'speed_rolling_mean_6', 'speed_rolling_mean_12', 'speed_rolling_std_6',
    'intensity_rolling_mean_6',
    'speed_diff', 'intensity_diff',
    'speed_to_max_ratio'
] + [col for col in df_featured.columns if col.startswith('segment_')]

# Targets
target_speed = 'AverageVehicleSpeed'
target_congestion = 'Congested'

# Create X and y
X = df_featured[feature_cols]
y_speed = df_featured[target_speed]
y_congestion = df_featured[target_congestion].astype(int)

# Train/Test split (80/20)
X_train, X_test, y_speed_train, y_speed_test, y_cong_train, y_cong_test = train_test_split(
    X, y_speed, y_congestion, test_size=0.2, random_state=42, shuffle=False
)

print(f"✅ Train set: {len(X_train)} records")
print(f"✅ Test set: {len(X_test)} records")
print(f"\n📊 Features: {len(feature_cols)}")
print(f"\n📋 Congestion distribution:")
print(f"  Train - Flowing: {(y_cong_train==0).sum()} | Congested: {(y_cong_train==1).sum()}")
print(f"  Test  - Flowing: {(y_cong_test==0).sum()} | Congested: {(y_cong_test==1).sum()}")

In [ ]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled")

## 🚀 STEP 5A: Train XGBoost (Congestion Classifier)

In [ ]:
import time

print("="*80)
print("🔥 TRAINING XGBOOST CLASSIFIER")
print("="*80)

start_time = time.time()

# Calculate scale_pos_weight for imbalanced data
scale_pos_weight = (y_cong_train == 0).sum() / (y_cong_train == 1).sum()
print(f"\n⚖️  Class imbalance ratio: {scale_pos_weight:.2f}")

# XGBoost model
xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    gamma=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    tree_method='hist'  # Fast training
)

# Train
xgb_model.fit(
    X_train_scaled, y_cong_train,
    eval_set=[(X_test_scaled, y_cong_test)],
    verbose=50
)

train_time = time.time() - start_time

# Predictions
y_pred = xgb_model.predict(X_test_scaled)
y_pred_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]

# Evaluation
print("\n" + "="*80)
print("📊 XGBOOST RESULTS")
print("="*80)
print(f"⏱️  Training time: {train_time:.1f} seconds")
print(f"\n{classification_report(y_cong_test, y_pred, target_names=['Flowing', 'Congested'])}")

# Feature importance (Top 15)
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False).head(15)

print("\n📈 Top 15 Important Features:")
print(feature_importance.to_string(index=False))

# Plot
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.xlabel('Importance')
plt.title('XGBoost Feature Importance (Top 15)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 🚀 STEP 5B: Train LightGBM (Speed Regressor)

In [ ]:
print("="*80)
print("🔥 TRAINING LIGHTGBM REGRESSOR")
print("="*80)

start_time = time.time()

# LightGBM model
lgb_model = lgb.LGBMRegressor(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_samples=20,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Train
lgb_model.fit(
    X_train_scaled, y_speed_train,
    eval_set=[(X_test_scaled, y_speed_test)],
    callbacks=[lgb.log_evaluation(50)]
)

train_time = time.time() - start_time

# Predictions
y_speed_pred = lgb_model.predict(X_test_scaled)

# Evaluation
mae = mean_absolute_error(y_speed_test, y_speed_pred)
rmse = np.sqrt(mean_squared_error(y_speed_test, y_speed_pred))
r2 = r2_score(y_speed_test, y_speed_pred)

print("\n" + "="*80)
print("📊 LIGHTGBM RESULTS")
print("="*80)
print(f"⏱️  Training time: {train_time:.1f} seconds")
print(f"\n📈 Metrics:")
print(f"  MAE:  {mae:.2f} km/h")
print(f"  RMSE: {rmse:.2f} km/h")
print(f"  R²:   {r2:.4f}")

# Scatter plot: Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_speed_test, y_speed_pred, alpha=0.3)
plt.plot([y_speed_test.min(), y_speed_test.max()], 
         [y_speed_test.min(), y_speed_test.max()], 'r--', lw=2)
plt.xlabel('Actual Speed (km/h)')
plt.ylabel('Predicted Speed (km/h)')
plt.title(f'LightGBM: Actual vs Predicted (MAE={mae:.2f}, R²={r2:.3f})')
plt.tight_layout()
plt.show()

## 🚀 STEP 5C: Train Prophet (Trend per Segment)

In [ ]:
print("="*80)
print("🔥 TRAINING PROPHET MODELS (Per Segment)")
print("="*80)

# Prepare Prophet data
prophet_models = {}
segments = df_featured['RefRoadSegment'].unique()

start_time = time.time()

for segment in segments:
    print(f"\n📊 Training Prophet for {segment}...")
    
    # Filter segment data
    df_segment = df_featured[df_featured['RefRoadSegment'] == segment].copy()
    
    # Prophet format
    df_prophet = pd.DataFrame({
        'ds': df_segment['DateObservedFrom'],
        'y': df_segment['AverageVehicleSpeed'],
        'intensity': df_segment['Intensity'],
        'is_weekend': df_segment['is_weekend'],
        'is_rush_hour': df_segment['is_rush_hour']
    })
    
    # Initialize Prophet
    model = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10
    )
    
    # Add regressors
    model.add_regressor('intensity')
    model.add_regressor('is_weekend')
    model.add_regressor('is_rush_hour')
    
    # Train
    model.fit(df_prophet)
    
    prophet_models[segment] = model
    print(f"  ✅ {segment} trained")

train_time = time.time() - start_time

print("\n" + "="*80)
print("📊 PROPHET RESULTS")
print("="*80)
print(f"⏱️  Total training time: {train_time:.1f} seconds")
print(f"✅ Trained {len(prophet_models)} models (one per segment)")

# Sample forecast visualization for one segment
sample_segment = segments[0]
sample_model = prophet_models[sample_segment]

# Forecast next 24 hours
future = sample_model.make_future_dataframe(periods=288, freq='5min')  # 288 = 24h / 5min
# Add regressors (use mean values for future)
future['intensity'] = df_featured[df_featured['RefRoadSegment']==sample_segment]['Intensity'].mean()
future['is_weekend'] = 0
future['is_rush_hour'] = 0

forecast = sample_model.predict(future)

# Plot
fig = sample_model.plot(forecast)
plt.title(f'Prophet Forecast - {sample_segment}')
plt.tight_layout()
plt.show()

# Components plot
fig = sample_model.plot_components(forecast)
plt.tight_layout()
plt.show()

## 💾 STEP 6: Save Models to Google Drive

In [ ]:
import os

# Create output directory
os.makedirs(MODEL_OUTPUT_PATH, exist_ok=True)

print("="*80)
print("💾 SAVING TRAINED MODELS")
print("="*80)

# Save XGBoost
xgb_path = os.path.join(MODEL_OUTPUT_PATH, 'xgboost_congestion.pkl')
joblib.dump(xgb_model, xgb_path)
print(f"✅ XGBoost saved: {xgb_path}")
print(f"   Size: {os.path.getsize(xgb_path) / 1024:.1f} KB")

# Save LightGBM
lgb_path = os.path.join(MODEL_OUTPUT_PATH, 'lightgbm_speed.pkl')
joblib.dump(lgb_model, lgb_path)
print(f"✅ LightGBM saved: {lgb_path}")
print(f"   Size: {os.path.getsize(lgb_path) / 1024:.1f} KB")

# Save Prophet (all segments)
prophet_path = os.path.join(MODEL_OUTPUT_PATH, 'prophet_models.pkl')
joblib.dump(prophet_models, prophet_path)
print(f"✅ Prophet saved: {prophet_path}")
print(f"   Size: {os.path.getsize(prophet_path) / 1024:.1f} KB")

# Save Scaler
scaler_path = os.path.join(MODEL_OUTPUT_PATH, 'scaler.pkl')
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved: {scaler_path}")
print(f"   Size: {os.path.getsize(scaler_path) / 1024:.1f} KB")

# Save feature columns
feature_cols_path = os.path.join(MODEL_OUTPUT_PATH, 'feature_columns.pkl')
joblib.dump(feature_cols, feature_cols_path)
print(f"✅ Feature columns saved: {feature_cols_path}")

print("\n" + "="*80)
print("🎉 ALL MODELS SAVED SUCCESSFULLY!")
print("="*80)
print("\n📋 NEXT STEPS:")
print("  1. Download models from Google Drive to local:")
print(f"     {MODEL_OUTPUT_PATH}")
print("  2. Place in: e:\\CĐTT2\\Smart-Transport\\...\\models\\saved_models\\")
print("  3. Load models in FastAPI for serving")
print("  4. Test predictions locally")

## 📊 STEP 7: Model Summary Report

In [ ]:
print("="*80)
print("📊 FINAL MODEL SUMMARY")
print("="*80)

summary = pd.DataFrame([
    {
        'Model': 'XGBoost Classifier',
        'Task': 'Congestion Detection',
        'Metric': f"{f1_score(y_cong_test, y_pred):.3f} F1-Score",
        'File': 'xgboost_congestion.pkl'
    },
    {
        'Model': 'LightGBM Regressor',
        'Task': 'Speed Prediction',
        'Metric': f"{mae:.2f} km/h MAE",
        'File': 'lightgbm_speed.pkl'
    },
    {
        'Model': f'Prophet ({len(prophet_models)} models)',
        'Task': 'Trend Forecasting',
        'Metric': 'Per-segment trend',
        'File': 'prophet_models.pkl'
    }
])

print(summary.to_string(index=False))
print("\n✅ Training completed successfully!")
print("📁 All models saved to Google Drive")
print("🚀 Ready for deployment")